In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import copy
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [68]:
df=pd.read_csv("training-v1/offenseval-training-v1.tsv",sep='\t')
df
    

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [3]:
tweets=df[["tweet"]]
labels = df[["subtask_a"]]
tweets1 = copy.deepcopy(tweets)

In [4]:

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt


In [5]:
tweets1['tidy_tweet'] = np.vectorize(remove_pattern)(tweets['tweet'], "@[\w]*") 
tweets1['tidy_tweet'] = tweets1['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
tweets1['tidy_tweet'] = tweets1['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
tokenized_tweet = tweets1['tidy_tweet'].apply(lambda x: x.split())
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 

In [6]:
vectorizer = TfidfVectorizer()
untokenized_data =[' '.join(tweet) for tweet in tokenized_tweet]
vectorizer = vectorizer.fit(untokenized_data)
vectors = vectorizer.transform(untokenized_data).toarray()


In [7]:
labels1 = labels['subtask_a'].values.tolist()

In [8]:
vectors1=vectors#this line was used for memory error debugging
labels1=labels#this line was used for memory error debugging
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors1, labels1, test_size=0.25)

In [9]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='auto', solver='newton-cg',)

In [10]:
classifier.fit(train_vectors, train_labels)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
f1_score1=f1_score(test_labels, classifier.predict(test_vectors), average='macro') 


In [12]:
print(f1_score1)

0.6763586869523308


In [13]:
print(confusion_matrix(test_labels, classifier.predict(test_vectors)))

[[2106  121]
 [ 670  413]]


### Using Average Tweet Length Instead of Whole Data

In [25]:
# tweets1['tidy_tweet'][0]
total_tweet = np.array([])
total_label = np.array([])
temp = []
counter = 0
for tweets in tweets1['tidy_tweet']:
    if len(tweets.split()) <= 22:
        total_tweet = np.append(total_tweet,tweets)
        total_label = np.append(total_label,labels.iloc[counter])
    counter += 1
#         total_tweet.append(tweets)

In [26]:
total_tweet

array([' She should ask a few native Americans what their take on this is ',
       '  Go home you re drunk     #MAGA #Trump          URL',
       ' Someone should veTaken  this piece of shit to a volcano    ',
       ..., ' And why report this garbage   We don t give a crap ',
       ' Pussy',
       '#Spanishrevenge vs  #justice #HumanRights and #FreedomOfExpression #Spain is a  #fakedemocracy                #cddr #shameonSpain #WakeupEurope  URL'],
      dtype='<U231')

In [33]:
tweets1 = pd.DataFrame({'tidy_tweet': total_tweet[:]})
# tweets1

In [38]:
labels = pd.DataFrame({'subtask_a': total_label[:]})
# labels

In [34]:
tokenized_tweet = tweets1['tidy_tweet'].apply(lambda x: x.split())
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 

In [35]:
vectorizer = TfidfVectorizer()
untokenized_data =[' '.join(tweet) for tweet in tokenized_tweet]
vectorizer = vectorizer.fit(untokenized_data)
vectors = vectorizer.transform(untokenized_data).toarray()


In [39]:
vectors1=vectors
labels1 = labels['subtask_a'].values.tolist()

In [40]:
vectors1=vectors#this line was used for memory error debugging
labels1=labels#this line was used for memory error debugging
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors1, labels1, test_size=0.25)

In [41]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='auto', solver='newton-cg',)

In [43]:
classifier.fit(train_vectors, train_labels)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
f1_score1=f1_score(test_labels, classifier.predict(test_vectors), average='macro') 


In [45]:
print(f1_score1)

0.6672120729482001


In [46]:
print(confusion_matrix(test_labels, classifier.predict(test_vectors)))

[[1441   35]
 [ 439  215]]


### Using Class Balancing Technique - Upsampling/Oversampling

In [50]:
df1=pd.read_csv("training-v1/offenseval-training-v1.tsv",sep='\t')
df1.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [54]:
df2 = df1.loc[df1.subtask_a == "OFF"]
df2

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN
...,...,...,...,...,...
13223,63482,@USER is advocating for conduct within bounds ...,OFF,TIN,GRP
13227,87416,@USER @USER @USER @USER Liars like the Antifa ...,OFF,TIN,GRP
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH


In [55]:
df3 =  pd.concat([df1, df2])

In [56]:
df3

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13223,63482,@USER is advocating for conduct within bounds ...,OFF,TIN,GRP
13227,87416,@USER @USER @USER @USER Liars like the Antifa ...,OFF,TIN,GRP
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH


In [57]:
tweets=df3[["tweet"]]
labels = df3[["subtask_a"]]
tweets1 = copy.deepcopy(tweets)

In [58]:

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt


In [59]:
tweets1['tidy_tweet'] = np.vectorize(remove_pattern)(tweets['tweet'], "@[\w]*") 
tweets1['tidy_tweet'] = tweets1['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
tweets1['tidy_tweet'] = tweets1['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
tokenized_tweet = tweets1['tidy_tweet'].apply(lambda x: x.split())
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 

In [60]:
vectorizer = TfidfVectorizer()
untokenized_data =[' '.join(tweet) for tweet in tokenized_tweet]
vectorizer = vectorizer.fit(untokenized_data)
vectors = vectorizer.transform(untokenized_data).toarray()


In [61]:
labels1 = labels['subtask_a'].values.tolist()

In [62]:
vectors1=vectors#this line was used for memory error debugging
labels1=labels#this line was used for memory error debugging
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors1, labels1, test_size=0.25)

In [63]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='auto', solver='newton-cg',)
classifier.fit(train_vectors, train_labels)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
f1_score1=f1_score(test_labels, classifier.predict(test_vectors), average='macro') 


In [65]:
print(f1_score1)

0.7651907444651175


In [66]:
print(confusion_matrix(test_labels, classifier.predict(test_vectors)))

[[1835  437]
 [ 594 1544]]


### Using No. Of Mentions

In [69]:
tweets=df[["tweet"]]
labels = df[["subtask_a"]]
tweets1 = copy.deepcopy(tweets)

In [70]:

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt


In [74]:
tweets1['tidy_tweet'] = np.vectorize(remove_pattern)(tweets['tweet'], "[\w]*") 
tweets1['tidy_tweet'] = tweets1['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
tweets1['tidy_tweet'] = tweets1['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
tokenized_tweet = tweets1['tidy_tweet'].apply(lambda x: x.split())
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 

In [75]:
tweets1

,tweet,tidy_tweet
0,@USER She should ask a few native Americans wh...,ntive Americns
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,
2,Amazon is investigating Chinese employees who ...,
3,"@USER Someone should'veTaken"" this piece of sh...",volcno
4,@USER @USER Obama wanted liberals &amp; illega...,
...,...,...
13235,@USER Sometimes I get strong vibes from people...,million murder dangerou
13236,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,
13237,@USER And why report this garbage. We don't g...,
13238,@USER Pussy,


In [76]:
vectorizer = TfidfVectorizer()
untokenized_data =[' '.join(tweet) for tweet in tokenized_tweet]
vectorizer = vectorizer.fit(untokenized_data)
vectors = vectorizer.transform(untokenized_data).toarray()


In [77]:
labels1 = labels['subtask_a'].values.tolist()

In [78]:
vectors1=vectors#this line was used for memory error debugging
labels1=labels#this line was used for memory error debugging
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors1, labels1, test_size=0.25)

In [79]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='auto', solver='newton-cg',)

In [80]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='auto', solver='newton-cg',)
classifier.fit(train_vectors, train_labels)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [81]:
f1_score1=f1_score(test_labels, classifier.predict(test_vectors), average='macro') 


In [82]:
print(f1_score1)

0.43684379332511275


In [83]:
print(confusion_matrix(test_labels, classifier.predict(test_vectors)))

[[2137   50]
 [1073   50]]
